<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/Honours_Repository/blob/main/SciBERT_train_single_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook documents the procedure for training the single-label models during the 2023 TRAM effort.

The `bootstrap-training-data` file contains the annotations that existed prior, as well as the annotations that were produced during the 2023 effort.

In [1]:
!pip install --upgrade --no-cache-dir gdown
#Upload the JSON file for fine-tuning (from cti-to-mitre-with-nlp)
import gdown

url = 'https://drive.google.com/file/d/1BCkEdKgmH49kjihmrxlXVUQvB0GsiQJW/view?usp=drive_link'
output_path = 'TRAM_fine_tuned_SciBERT.json'
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1BCkEdKgmH49kjihmrxlXVUQvB0GsiQJW
To: /content/TRAM_fine_tuned_SciBERT.json
100%|██████████| 1.38M/1.38M [00:00<00:00, 74.5MB/s]


'TRAM_fine_tuned_SciBERT.json'

In [2]:
import pandas as pd
import json



with open('TRAM_fine_tuned_SciBERT.json') as f:
    data = json.loads(f.read())

data = pd.DataFrame(
    [
        {'text': row['text'], 'label': row['label']}
        for row in data

    ]
)
print(data)

                                                   text      label
0     Anchor has used cmd.exe to run its self deleti...  T1059.003
1     Zeus Panda can launch an interface where it ca...  T1059.003
2     Chimera has used the Windows Command Shell and...  T1059.003
3     Cuba has used cmd.exe /c and batch files for e...  T1059.003
4     MechaFlounder has the ability to run commands ...  T1059.003
...                                                 ...        ...
9806                           Kazuar can delete files.  T1070.004
9807  Cobalt Strike can exploit vulnerabilities such...      T1068
9808  QakBot can send stolen information to C2 nodes...      T1041
9809  Turla RPC backdoors have also searched for fil...      T1083
9810  Ramsay has created Registry Run keys to establ...  T1547.001

[9811 rows x 2 columns]


In [3]:
!pip install transformers torch

We then load the model and move it to the GPU.

In [4]:
import transformers
import torch

mode: 'bert or gpt' = 'bert'
cuda = torch.device('cuda')

if mode == 'bert':
    model = transformers.BertForSequenceClassification.from_pretrained(
        "allenai/scibert_scivocab_uncased",
        num_labels=data['label'].nunique(),
        output_attentions=False,
        output_hidden_states=False,
    )
    tokenizer = transformers.BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased", max_length=512)
elif mode == 'gpt':
    model = transformers.GPT2ForSequenceClassification.from_pretrained(
        "gpt2",
        num_labels=data['label'].nunique(),
        output_attentions=False,
        output_hidden_states=False,
    )
    tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2", max_length=512)
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
else:
    raise ValueError(f"mode must be one of bert or gpt, but is {mode = !r}")

model.train().to(cuda)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

We will represent the labels using one hot encoding.

The `apply_attention_mask` function returns an attention mask (which is a tensor) where the element for every non-padding token is `1`.

In [5]:
from sklearn.preprocessing import OneHotEncoder as OHE

encoder = OHE(sparse_output=False)
encoder.fit(data[['label']])

def tokenize(samples: 'list[str]'):
    return tokenizer(samples, return_tensors='pt', padding='max_length', truncation=True, max_length=512).input_ids

def load_data(x, y, batch_size=10):
    x_len, y_len = x.shape[0], y.shape[0]
    assert x_len == y_len
    for i in range(0, x_len, batch_size):
        slc = slice(i, i + batch_size)
        yield x[slc].to(cuda), y[slc].to(cuda)

def apply_attention_mask(x):
    return x.ne(tokenizer.pad_token_id).to(int)


In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=.2, stratify=data['label'])

x_train = tokenize(train['text'].tolist())
x_train

tensor([[  102, 21159, 22229,  ...,     0,     0,     0],
        [  102, 16660,  5688,  ...,     0,     0,     0],
        [  102, 20600,   434,  ...,     0,     0,     0],
        ...,
        [  102,  1365, 29887,  ...,     0,     0,     0],
        [  102, 25179,   282,  ...,     0,     0,     0],
        [  102,  1109, 11277,  ...,     0,     0,     0]])

In [7]:
print(len(x_train))

7848


In [8]:
y_train = torch.Tensor(encoder.transform(train[['label']]))
y_train

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [9]:
x_test = tokenize(test['text'].tolist())
y_test = torch.Tensor(encoder.transform(test[['label']]))

The hyperparameters shown here are those that we used, including the number of epochs and batch size.

In [10]:
from torch.optim import AdamW
from tqdm import tqdm
from statistics import mean

optim = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

for epoch in range(6):
    epoch_losses = []
    validation_loss = []
    model.train()
    for x, y in tqdm(load_data(x_train, y_train, batch_size=10)):
        model.zero_grad()
        out = model(x, attention_mask=apply_attention_mask(x), labels=y)
        epoch_losses.append(out.loss.item())
        out.loss.backward()
        optim.step()
    print(f"epoch {epoch + 1} loss: {mean(epoch_losses)}")
    # Free up GPU memory
    torch.cuda.empty_cache()
    model.eval()
    for x, y in load_data(x_test, y_test, batch_size=10):

    # validation data
        validation_output = model(x, attention_mask=apply_attention_mask(x), labels=y)
        validation_loss.append(validation_output.loss.item())

    print(f"Validation for epoch {epoch + 1} loss: {mean(validation_loss)}")
    # Free up GPU memory
    torch.cuda.empty_cache()


785it [11:33,  1.13it/s]


epoch 1 loss: 0.12712413937233055
Validation for epoch 1 loss: 0.09163273617549596


785it [11:33,  1.13it/s]


epoch 2 loss: 0.07100867721590266
Validation for epoch 2 loss: 0.05098084413309388


785it [11:33,  1.13it/s]


...

epoch 4 loss: 0.02330432324106716
Validation for epoch 4 loss: 0.025906124336616643


785it [11:33,  1.13it/s]


epoch 5 loss: 0.014950301550376188
Validation for epoch 5 loss: 0.02196513876545853


785it [11:33,  1.13it/s]


epoch 6 loss: 0.009599128989562108
Validation for epoch 6 loss: 0.023518645121871972


In [11]:
import torch.nn.functional as F

model.eval()

preds = []
batch_size = 20

x_test = tokenize(test['text'].tolist())

with torch.no_grad():
    for i in range(0, x_test.shape[0], batch_size):
        x = x_test[i : i + batch_size].to(cuda)
        out = model(x, attention_mask=apply_attention_mask(x))
        preds.extend(out.logits.to('cpu'))

predicted_labels = (
    encoder.inverse_transform(
        F.one_hot(
            torch.vstack(preds).softmax(-1).argmax(-1),
            num_classes=50
        )
        .numpy()
    )
    .reshape(-1)
)

predicted_labels

array(['T1566.001', 'T1053.005', 'T1110', ..., 'T1573.001', 'T1005',
       'T1083'], dtype=object)

In [12]:
from sklearn.metrics import precision_recall_fscore_support as calculate_score

predicted = list(predicted_labels)
actual = test['label'].tolist()

labels = sorted(data['label'].unique())

scores = calculate_score(actual, predicted, labels=labels)

scores_df = pd.DataFrame(scores).T
scores_df.columns = ['P', 'R', 'F1', '#']
scores_df.index = labels
scores_df.loc['(micro)'] = calculate_score(actual, predicted, average='micro', labels=labels)
scores_df.loc['(macro)'] = calculate_score(actual, predicted, average='macro', labels=labels)

scores_df

,P,R,F1,#
T1003.001,0.888889,0.666667,0.761905,36.0
T1005,0.833333,0.681818,0.750000,22.0
T1012,0.888889,0.842105,0.864865,38.0
T1016,0.975000,0.780000,0.866667,50.0
T1021.001,0.827586,0.727273,0.774194,33.0
T1027,0.792453,0.700000,0.743363,60.0
T1033,0.941176,0.864865,0.901408,37.0
T1036.005,0.718750,0.741935,0.730159,62.0
T1041,0.829268,0.871795,0.850000,39.0
T1047,0.740741,0.909091,0.816327,22.0


In [13]:
model = model.save_pretrained("scibert_model")
tokenizer = tokenizer.save_pretrained("scibert_tokenizer")

In [ ]:
!zip -r scibert_model.zip scibert_model/
!zip -r scibert_tokenizer.zip scibert_tokenizer/
from google.colab import files
files.download('scibert_model.zip')
files.download('scibert_tokenizer.zip')